#  Genomic regions for Kircher _et al_ 2019 for _in-silico_ mutagenesis

For the saturation mutagenesis via MPRA assays in [Kircher _et al_ (2019)](https://doi.org/10.1038/s41467-019-11526-w), we determine for which parts (in the form of genomic regions) of their 21 regions we are missing allelic effect predictions.

## Setup

### Imports

In [3]:
import polars as pl
import numpy as np
import pandas as pd
import duckdb
from pathlib import Path

### Paths to data and databases

In [4]:
PROJECT_ROOT = Path('/hpc/group/igvf')
DATA_ROOT = Path("../../igvf-pm")
DB_ROOT = PROJECT_ROOT / 'db'
MUT_PRED_DB = DB_ROOT / 'cCRE-preds-K562'
DATA_DIR = PROJECT_ROOT / 'benchmarks' / 'Kircher_et_al_2019'

## Create region maps

### Parse Kircher _et al_ (2019) data files

In principle we could parse the region coordinates from the spreadsheets of enhancer and promoter regions for the paper. Instead we parse the data files directly, which include all chromosomal positions.

The columns in the files are the following (according to, and taken verbatim from, the [wiki accompanying the file deposition](https://osf.io/75b2m/wiki/Files/)):
1. **Chromosome** - Chromosome of the variant.
2. **Position** - Chromosomal position (GRCh38 or GRCh37) of the variant. _(We are using the GRCh38 coordinates)_
3. **Ref** - Reference allele of the variant (A, T, G, or C).
4. **Alt** - Alternative allele of the variant (A, T, G, or C). One base-pair deletions are represented as -.
5. **Tags** - Number of unique tags associated with the variant.
6. **DNA** - Count of DNA sequences that contain the variant (used for fitting the linear model).
7. **RNA** - Count of RNA sequences that contain the variant (used for fitting the linear model).
8. **Value** - Log2 variant expression effect derived from the fit of the linear model (coefficient).
9. **P-Value** - P-value of the coefficient.

So we can distinguish between enhancer and promoter regions, we'll include the filenames and parse them.

In [5]:
mut_data = pl.scan_csv(DATA_DIR / "*" / "*.tsv", separator="\t",
                       has_header=True, null_values=["NA"], include_file_paths="file_path")

For parsing the file paths, the directory indicates the type of region (enhancer or promoter, with the UCE being lumped in with enhancers). The filenames (i.e., base names) themselves have a prefix signifying the genome assembly version, the file extension as a suffix, and the part in between being the experiment. Most experiments correspond 1:1 to regions, but some include replicates, reverse strand, different protocols (time of harvesting, for example), etc. It appears that designations of replicate, protocol, etc are appended following either a dot or a dash.

In [6]:
mut_data_proc = mut_data.with_columns(
    pl.col('Position').add(-1).alias('allele_pos'),
    (pl.lit('chr') + pl.col('Chromosome')).alias('chrom'),
    pl.col('file_path').str.extract(r'GRCh38_(.*).tsv$').alias('experiment'),
    pl.col('file_path').str.split('/').list.get(-2).alias('region_type')
).with_columns(
    pl.col('experiment').str.extract(r'^([^-.]+)').alias('region'),
).select(['chrom', 'allele_pos', 'experiment', 'region', 'region_type']).drop_nulls()

### Table of Kircher _et al_ regions

In [7]:
kircher_regions = duckdb.sql(
    "select region_type, region, chrom, "
    "min(allele_pos) as start_pos, max(allele_pos) as end_pos, "
    "max(allele_pos) - min(allele_pos) + 1 as region_length, "
    "group_concat(distinct experiment, ',') as experiments "
    "from mut_data_proc group by region_type, region, chrom "
    "order by region_type, region, chrom")
kircher_regions

┌─────────────┬───────────────┬─────────┬───────────┬───────────┬───────────────┬─────────────────────────────────────┐
│ region_type │    region     │  chrom  │ start_pos │  end_pos  │ region_length │             experiments             │
│   varchar   │    varchar    │ varchar │   int64   │   int64   │     int64     │               varchar               │
├─────────────┼───────────────┼─────────┼───────────┼───────────┼───────────────┼─────────────────────────────────────┤
│ Enhancers   │ BCL11A        │ chr2    │  60494939 │  60495538 │           600 │ BCL11A                              │
│ Enhancers   │ IRF4          │ chr6    │    396142 │    396592 │           451 │ IRF4                                │
│ Enhancers   │ IRF6          │ chr1    │ 209815789 │ 209816389 │           601 │ IRF6                                │
│ Enhancers   │ MYCrs11986220 │ chr8    │ 127519269 │ 127519731 │           463 │ MYCrs11986220                       │
│ Enhancers   │ MYCrs6983267  │ chr8    

### Map table of Kircher _et al_ regions to predictions for cCRE regions

In [8]:
mutpreds = duckdb.read_parquet(str(DB_ROOT / f'{MUT_PRED_DB}/**/*.parquet'), hive_partitioning=True)

In [9]:
kircher_cre_map = duckdb.sql(
    "select k.region_type, k.region, k.chrom, k.start_pos, k.end_pos, k.region_length, "
    "m.cre_start, m.cre_end, min(allele_pos) as pred_start, max(allele_pos) as pred_end, "
    "max(allele_pos) - min(allele_pos) + 1 as pred_length, "
    "round(pred_length * 100 / region_length, 1) as region_cov "
    "from kircher_regions k left join mutpreds m "
    "on (k.chrom = m.chrom and k.start_pos <= m.allele_pos and k.end_pos >= m.allele_pos) "
    "group by k.region_type, k.region, k.chrom, k.start_pos, k.end_pos, k.region_length, m.cre_start, m.cre_end "
    "order by k.region_type, k.region, k.chrom"
).df().astype({f: 'Int64' for f in ['cre_start','cre_end','pred_start','pred_end','pred_length']})
kircher_cre_map

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,region_type,region,chrom,start_pos,end_pos,region_length,cre_start,cre_end,pred_start,pred_end,pred_length,region_cov
0,Enhancers,BCL11A,chr2,60494939,60495538,600,60495301,60495562,60495301,60495538,238,39.7
1,Enhancers,IRF4,chr6,396142,396592,451,395920,396157,396142,396156,15,3.3
2,Enhancers,IRF4,chr6,396142,396592,451,396163,396513,396188,396487,300,66.5
3,Enhancers,IRF6,chr1,209815789,209816389,601,209815503,209815835,209815789,209815818,30,5.0
4,Enhancers,IRF6,chr1,209815789,209816389,601,209815914,209816264,209815939,209816238,300,49.9
5,Enhancers,MYCrs11986220,chr8,127519269,127519731,463,<NA>,<NA>,<NA>,<NA>,<NA>,NaN
6,Enhancers,MYCrs6983267,chr8,127400828,127401427,600,127400782,127401030,127400828,127401029,202,33.7
7,Enhancers,MYCrs6983267,chr8,127400828,127401427,600,127401052,127401303,127401052,127401302,251,41.8
8,Enhancers,MYCrs6983267,chr8,127400828,127401427,600,127401374,127401609,127401374,127401427,54,9.0
9,Enhancers,RET,chr10,43086478,43087077,600,43086666,43086842,43086666,43086841,176,29.3


### Table of Kircher _et al_ regions matching cCRE predictions

For each Kircher _et al_ region, aggregate the number of cCREs matching, the total number of positions with predictions, and the cumulative percent of the Kircher region covered by the predictions.

In [10]:
kircher_matching_cres = duckdb.sql(
    "select region_type, region, concat(chrom, ':', start_pos, '-', end_pos+1) as region_id, "
    "region_length, "
    "count(distinct cre_start) as total_cres, "
    "sum(pred_length) as total_pred_pos, "
    "round(sum(region_cov), 3) as total_region_cov "
    "from kircher_cre_map "
    "where pred_length is not null "
    "group by region_type, region, region_id, region_length "
    "order by region_type, region"
).df().astype({'total_pred_pos': int})
kircher_matching_cres

,region_type,region,region_id,region_length,total_cres,total_pred_pos,total_region_cov
0,Enhancers,BCL11A,chr2:60494939-60495539,600,1,238,39.7
1,Enhancers,IRF4,chr6:396142-396593,451,2,315,69.8
2,Enhancers,IRF6,chr1:209815789-209816390,601,2,330,54.9
3,Enhancers,MYCrs6983267,chr8:127400828-127401428,600,3,507,84.5
4,Enhancers,RET,chr10:43086478-43087078,600,3,396,66.0
5,Enhancers,SORT1,chr1:109274650-109275251,601,2,369,61.4
6,Enhancers,UC88,chr2:161238407-161238998,591,3,415,70.2
7,Enhancers,ZFAND3,chr6:37807498-37808077,579,1,300,51.8
8,Enhancers,ZRSh,chr7:156791118-156791604,486,1,243,50.0
9,Promoters,FOXE1,chr9:97853254-97853854,600,3,478,79.6


### Table of Kircher _et al_ regions not matching cCREs

In [11]:
kircher_not_matching = duckdb.sql(
    "select region_type, region, "
    "concat(chrom, ':', start_pos, '-', end_pos+1) as region_id, region_length "
    "from kircher_cre_map "
    "where pred_length is null").df()
kircher_not_matching

,region_type,region,region_id,region_length
0,Enhancers,MYCrs11986220,chr8:127519269-127519732,463
1,Enhancers,TCF7L2,chr10:112998239-112998839,600
2,Promoters,F9,chrX:139530462-139530765,303
3,Promoters,HBB,chr11:5227021-5227208,187
4,Promoters,HBG1,chr11:5249804-5250078,274


### Write out region descriptor files

We'll divide this into the following region descriptors for running allelic effect predictions (through _in-silico_ mutagenesis), where a region descriptor is an ID of the form `chr:start-end`, with the interval in half-open zero-based coordinates. (I.e., _start_ is the zero-based coordinate for the first position for which predictions are needed, _end_ is the zero-based coordinate position _after_ the last position for which predictions are needed, and the length of the region for which predictions are needed is therefore _end-start_.)

* Kircher regions not matching any cCRE region for which we have predictions ("_not\_matching_")
* For Kircher regions matching one or more cCRE regions for which we have predictions:
  - The part of the Kircher regions _prior_ to their _first_ cCRE matches ("_pre\_cre_");
  - The part of the Kircher regions _after_ their last cCRE matches ("_post\_cre_");
  - The part of the Kircher regions _between_ two consecutive cCREs that they matche ("_inter\_cre_");
  - The above three combined together ("_all\_missing_").

Note that these region parts are the ones _missing_ predictions, compared to the cCRE predictions from _in-silico_ mutagenesis. If there are no cCRE predictions to start from (e.g., for a new model), we also generate a list of full Kircher regions matching cCREs ("_cres\_full_"), which would be used in complement to the regions not matching any cCRE.

#### Regions that don't match cCREs at all

In [12]:
kircher_not_matching.loc[:,['region_id','region_length']].to_csv(
    DATA_DIR / 'kircher_not_matching.txt', sep='\t', index=False, header=False)

#### Full regions that partially or fully match cCREs

If one needs to run predictions for the entirety of the Kircher regions, this together with the regions not matching any cCRE would be the two files.

In [13]:
kircher_matching_cres.loc[:,['region_id','region_length']].to_csv(
    DATA_DIR / 'kircher_matching_cres_full.txt', sep='\t', index=False, header=False)

#### Parts of regions _prior_ to where they match cCREs

In [14]:
pre_cre = duckdb.sql(
    "select region_type, region, concat(chrom, ':', start_pos, '-', min(pred_start)) as region_id, "
    "min(pred_start) - start_pos as region_length "
    "from kircher_cre_map "
    "where pred_length is not null "
    "group by region_type, region, chrom, start_pos "
    "having min(pred_start) > start_pos"
)
pre_cre.df().loc[:,['region_id','region_length']].to_csv(
    DATA_DIR / 'kircher_matching_pre_cre.txt', sep='\t', index=False, header=False)
pre_cre

┌─────────────┬─────────┬──────────────────────────┬───────────────┐
│ region_type │ region  │        region_id         │ region_length │
│   varchar   │ varchar │         varchar          │     int64     │
├─────────────┼─────────┼──────────────────────────┼───────────────┤
│ Enhancers   │ BCL11A  │ chr2:60494939-60495301   │           362 │
│ Enhancers   │ ZFAND3  │ chr6:37807498-37807632   │           134 │
│ Enhancers   │ ZRSh    │ chr7:156791118-156791361 │           243 │
│ Promoters   │ GP1BA   │ chr22:19723265-19723293  │            28 │
│ Promoters   │ PKLR    │ chr1:155301394-155301406 │            12 │
│ Promoters   │ LDLR    │ chr19:11089230-11089248  │            18 │
│ Promoters   │ MSMB    │ chr10:46046243-46046273  │            30 │
└─────────────┴─────────┴──────────────────────────┴───────────────┘

#### Parts of regions _after_ the cCREs where they match

In [15]:
post_cre = duckdb.sql(
    "select region_type, region, concat(chrom, ':', max(pred_end), '-', end_pos+1) as region_id, "
    "end_pos + 1 - max(pred_end) as region_length "
    "from kircher_cre_map "
    "where pred_length is not null "
    "group by region_type, region, chrom, end_pos "
    "having max(pred_end) < end_pos"
)
post_cre.df().loc[:,['region_id','region_length']].to_csv(
    DATA_DIR / 'kircher_matching_post_cre.txt', sep='\t', index=False, header=False)
post_cre

┌─────────────┬─────────┬──────────────────────────┬───────────────┐
│ region_type │ region  │        region_id         │ region_length │
│   varchar   │ varchar │         varchar          │     int64     │
├─────────────┼─────────┼──────────────────────────┼───────────────┤
│ Enhancers   │ IRF4    │ chr6:396487-396593       │           106 │
│ Enhancers   │ IRF6    │ chr1:209816238-209816390 │           152 │
│ Promoters   │ GP1BA   │ chr22:19723592-19723650  │            58 │
│ Promoters   │ MSMB    │ chr10:46046572-46046834  │           262 │
│ Promoters   │ PKLR    │ chr1:155301705-155301864 │           159 │
│ Enhancers   │ SORT1   │ chr1:109275116-109275251 │           135 │
│ Enhancers   │ ZFAND3  │ chr6:37807931-37808077   │           146 │
│ Promoters   │ HNF4A   │ chr20:44355753-44355804  │            51 │
└─────────────┴─────────┴──────────────────────────┴───────────────┘

#### Parts of regions in between cCREs that they match

This will only match Kircher regions that match multiple (2 or 3) consecutive cCREs. 

In [16]:
inter_cre = duckdb.sql(
    "select k1.region_type, k1.region, "
    "concat(k1.chrom, ':', k1.pred_end+1, '-', k2.pred_start) as region_id, "
    "k2.pred_start - (k1.pred_end + 1) as region_length "
    "from kircher_cre_map k1, kircher_cre_map k2 "
    "where k1.region = k2.region and k1.pred_start < k2.pred_start "
    "and not exists ("
    "select 1 from kircher_cre_map k3 "
    "where k3.region = k1.region and "
    "k3.pred_start > k1.pred_start and k3.pred_start < k2.pred_start) "
    "order by k1.region_type, k1.region"
)
inter_cre.df().loc[:,['region_id','region_length']].to_csv(
    DATA_DIR / 'kircher_matching_inter_cre.txt', sep='\t', index=False, header=False)
inter_cre

┌─────────────┬──────────────┬──────────────────────────┬───────────────┐
│ region_type │    region    │        region_id         │ region_length │
│   varchar   │   varchar    │         varchar          │     int64     │
├─────────────┼──────────────┼──────────────────────────┼───────────────┤
│ Enhancers   │ IRF4         │ chr6:396157-396188       │            31 │
│ Enhancers   │ IRF6         │ chr1:209815819-209815939 │           120 │
│ Enhancers   │ MYCrs6983267 │ chr8:127401030-127401052 │            22 │
│ Enhancers   │ MYCrs6983267 │ chr8:127401303-127401374 │            71 │
│ Enhancers   │ RET          │ chr10:43086662-43086666  │             4 │
│ Enhancers   │ RET          │ chr10:43086842-43087042  │           200 │
│ Enhancers   │ SORT1        │ chr1:109274719-109274817 │            98 │
│ Enhancers   │ UC88         │ chr2:161238465-161238590 │           125 │
│ Enhancers   │ UC88         │ chr2:161238890-161238941 │            51 │
│ Promoters   │ FOXE1        │ chr9:97

#### All non-matched parts combined for regions that partially match cCREs

In [17]:
pre_cre.union(post_cre).union(inter_cre).df().loc[:,['region_id','region_length']].to_csv(
    DATA_DIR / 'kircher_matching_all_missing.txt', sep='\t', index=False, header=False)